### Part 1.11 - Extract_data_55000_movies

#### Import the libraries

In [ ]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook
# Example of Use: tqdm_notebook(examples, desc="Converting examples to features")

#### Eleventh 5000 movies (55,000)

Extract data about:

* field 1: Plot Summary
* field 2: Actors
* field 3: Directors
* field 4: IMDB rating
* field 5: Plot Synopsis
* field 6: Reviews

In [ ]:
"""
Open the movie_content urls for the eleventh 5000 movies and create the souplist of those URLs
"""
content_eleven = []

with open(os.path.join(os.getcwd(), 'movie_content_url\\data_eleven_14012020.pkl'), 'rb') as f:
    
    content_eleven = pickle.load(f)

print("Number of URLs: {}".format(len(content_eleven)))

content_souplist_eleven = []

for i in tqdm_notebook(content_eleven):
    
    content_souplist_eleven.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_eleven)))

In [ ]:
"""
Open the synopsis urls of the eleventh 5000 movies and create the souplist of those URLs
"""
synopsis_eleven = []

with open(os.path.join(os.getcwd(), 'synopsis_url\\synopsis_eleven_14012020.pkl'), 'rb') as f:
    
    synopsis_eleven = pickle.load(f)

print("Number of URLs: {}".format(len(synopsis_eleven)))

synopsis_souplist_eleven = []

for i in tqdm_notebook(synopsis_eleven):
    
    synopsis_souplist_eleven.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(synopsis_souplist_eleven)))

In [ ]:
"""
Open the reviews urls of the eleventh 5000 movies and create the souplist of those URLs
"""
review_eleven = []

with open(os.path.join(os.getcwd(), 'reviews_url\\review_eleven_16012020.pkl'), 'rb') as f:
    
    review_eleven = pickle.load(f)

print("Number of URLs: {}".format(len(review_eleven)))

review_souplist_eleven = []

for i in tqdm_notebook(review_eleven):
    
    review_souplist_eleven.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_eleven)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_eleven)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_eleven)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_eleven)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name if len(item)!=0]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_eleven)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract plot synopsis

Having extracted the synopsis content for each movie, we noticed that roughly more than 50% percent of the movies have a synopsis text.
Thus, in order not to erase almost half of the first 5,000 we will not further proceed in keeping the synopsis text
"""
synopsis_step_one = []
synopsis_step_two = []
synopsis_step_three = []

[synopsis_step_one.append(i.find_all('ul', {'class':'ipl-zebra-list', 'id':'plot-synopsis-content'})) for i in tqdm_notebook(synopsis_souplist_eleven)]

[[synopsis_step_two.append(j.find_all('li', {'class':'ipl-zebra-list__item'})) for j in i] for i in synopsis_step_one]

[[synopsis_step_three.append(j.text.strip(' ').replace('\n', '').replace('\\', '')) for j in i] for i in synopsis_step_two]

index_to_remove_no_synopsis = [i for i,x in enumerate(synopsis_step_one) if not x]

print("Length of Synopsis list: {}".format(len(synopsis_step_three)))
print("Length of the list with Movies that don't have synopsis: {}".format(len(index_to_remove_no_synopsis)))
if len(index_to_remove_no_synopsis) == 0:
    print("None of the movies miss a synopsis")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_synopsis))

In [ ]:
"""
Print the movies with no synopsis text.
Result: We observed that many movies missed a synopsis text. Thus, we decided to not use synopsis for furhter analysis (although will be extracted)
"""
print([i for i,x in enumerate(synopsis_step_one) if not x])
print([i for i,x in enumerate(synopsis_step_two) if not x])
print([i for i,x in enumerate(synopsis_step_three) if not x])

# Many of the movies do not have a synopsis text, this is it won't be used for furhter analysis (althpugh will be extracted)

In [ ]:
"""
Field 6: Extract movie reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_eleven)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
"""
Print the movies with no user reviews text.
"""
print([i for i,x in enumerate(myfield_review_step_one) if not x])
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

<b> - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - </b>

#### Dataframe creation based on the movie content, synopsis and reviews extracted

In [ ]:
"""
Import the movie_content (the extracted HTML document of the column imdb_url)
Import the reviews content (the extracted HTML document of the column reviews_url)
"""
with open(os.path.join(os.getcwd(), 'movie_content_url\\data_eleven_14012020.pkl'), 'rb') as f:
    
    content_eleven = pickle.load(f)

print("Number of URLs: {}".format(len(content_eleven)))

with open(os.path.join(os.getcwd(), 'reviews_url\\review_eleven_16012020.pkl'), 'rb') as f:
    
    review_eleven = pickle.load(f)

print("Number of URLs: {}".format(len(review_eleven)))

In [ ]:
"""
Below we import the 5 fields extracted previously
However, we we imported the "rating" file, we noticed that they did not downloaded properly. Actually, the respective list was empty,
thus we decided to download again the data in order no to make any other mistake with the index of the movies.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\pre-indexed files\\plot_eleven_18012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\pre-indexed files\\rating_eleven_18012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\pre-indexed files\\actors_eleven_18012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\pre-indexed files\\director_eleven_18012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\pre-indexed files\\synopsis_eleven_18012020.pkl'), 'rb') as f:
    
    synopsis = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\pre-indexed files\\reviews_eleven_18012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
print(len(plot))
print(len(rating))
print(len(actors))
print(len(director))
print(len(synopsis))
print(len(reviews))

print(rating) #empty

In [ ]:
"""
Running the commands below we can spot the number of indexes that should be removed from the data extracted.
"Add a Plot": Denoted that the movies does have a written plot on its IMDB website.
"It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.": Also denotes a non-written synopsis.

Synopsis indexes are not further considered. There are too many movies with no synopsis text and there is no poin in erasing almost half of the first 5000 movies due to missing synopsis text.
"""
matching_add_plot = [s for s, x in enumerate(plot) if "Add a Plot" in x]

print("Number of movies with 'Add Plot': ", len(matching_add_plot), '\n')

matching_add_synopsis = [s for s, x in enumerate(synopsis) if 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.' in x]

print("Number of movies with no synopsis text: ", len(matching_add_synopsis))

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[4,5,15,20,24,37,105,121,128,195,236,271,321,322,323,324,361,366,375,376,386,414,415,416,455,504,528,532,612,631,632,633,634,635,724,752,874,963,982,1031,1073,1119,1145,1164,1178,1226,1227,1228,1229,1230,1273,1282,1295,1296,1330,1348,1363,1464,1470,1473,1837,1838,1841,1928,2068,2073,2074,2076,2077,2091,2092,2094,2107,2123,2149,2180,2194,2243,2268,2277,2281,2283,2312,2315,2321,2327,2501,2534,2593,2636,2698,2738,2759,2761,2864,2875,3008,3021,3022,3027,3031,3034,3046,3047,3051,3056,3058,3061,3071,3072,3073,3074,3075,3076,3077,3080,3081,3084,3087,3089,3090,3091,3092,3093,3094,3095,3096,3119,3121,3134,3145,3190,3226,3227,3228,3229,3230,3231,3233,3285,3316,3332,3458,3519,3520,3521,3527,3586,3621,3666,3739,3759,3770,3787,3809,3832,3842,3938,3942,3960,4010,4039,4048,4081,4083,4120,4129,4190,4222,4223,4227,4284,4306,4317,4335,4429,4454,4464,4522,4531,4596,4629,4638,4642,4671,4697,4698,4737,4757,4788,4795,4920,4921,4926,4927,4928,4933,4963,4964,4976,4977,4995,874,1031,1079,1273,1932,2177,2269,2283,2336,2419,3455,3581,3740,4585,4834,612,874,1031,2281,2698,3008,14,18,22,44,66,84,146,149,166,171,207,244,312,322,323,324,353,412,433,455,456,537,586,592,593,624,635,705,735,747,843,932,937,993,1012,1031,1056,1072,1078,1144,1199,1229,1256,1331,1378,1484,1511,1512,1513,1514,1515,1516,1517,1518,1689,1739,1877,1912,1931,1936,1987,2039,2099,2104,2149,2154,2160,2172,2274,2285,2311,2335,2341,2345,2398,2404,2418,2446,2452,2544,2547,2599,2613,2615,2620,2622,2623,2631,2640,2650,2668,2670,2671,2672,2676,2680,2682,2718,2746,2774,2794,3121,3167,3174,3220,3221,3296,3320,3322,3345,3389,3425,3457,3470,3485,3486,3501,3518,3549,3556,3570,3580,3589,3592,3613,3659,3689,3730,3739,3758,3759,3772,3872,3874,3927,3947,3953,4008,4018,4036,4038,4120,4127,4148,4285,4297,4339,4404,4418,4419,4420,4447,4452,4472,4473,4476,4539,4549,4554,4646,4655,4715,4762,4763,4771,4786,4917,4926,4958,4963,4966,4988]
index_remove = list(dict.fromkeys(index_remove))
print(len(index_remove))
content_index_eleven = [i for j, i in enumerate(content_eleven) if j not in index_remove]
review_index_eleven = [i for j, i in enumerate(review_eleven) if j not in index_remove]
print(len(content_index_eleven))
print(len(review_index_eleven))

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_eleven\\content_index_eleven_20012020.pkl'), 'wb') as f:  
#     pickle.dump(content_index_eleven, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_eleven\\review_index_eleven_20012020.pkl'), 'wb') as f:
#     pickle.dump(review_index_eleven, f)

In [ ]:
"""
Import the pruned list of movie_content & reviews web pages. The indexes that have been removed did not contain one or more -
of the following fields: plot, actors, director(s), imdb_rating, review(s)

It is important to re-run the process of extracting the information in order to assure that we have extracted the correct information and text for each movie
Otherwise, we might observe mismatches in the fields assigned to each movie title.
"""
content_souplist_eleven = []

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\content_index_eleven_20012020.pkl'), 'rb') as f:
    
    content_index_eleven = pickle.load(f)

print("Number of URLs: {}".format(len(content_index_eleven)))

for i in tqdm_notebook(content_index_eleven):
    
    content_souplist_eleven.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_eleven)))

#-----------------------------------------------------------------------------------------------------

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\review_index_eleven_20012020.pkl'), 'rb') as f:
    
    review_eleven = pickle.load(f)

print("Number of URLs: {}".format(len(review_eleven)))

review_souplist_eleven = []

for i in tqdm_notebook(review_eleven):
    
    review_souplist_eleven.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_eleven)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_eleven)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_eleven)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_eleven)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_eleven)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract movie user reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_eleven)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
print(len([i for i,x in enumerate(myfield_review_step_one) if not x]))

print([i for i,x in enumerate(myfield_review_step_two) if not x])

print([i for i,x in enumerate(myfield_review_step_three) if not x])

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# # Pickle the requests file for further use!

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\plot_eleven_20012020.pkl'), 'wb') as f: 
#     pickle.dump(plot_summary, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\actors_eleven_20012020.pkl'), 'wb') as f:
#     pickle.dump(actors_list, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\director_eleven_20012020.pkl'), 'wb') as f:
#     pickle.dump(director_names, f)

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\reviews_eleven_24012020.pkl'), 'wb') as f:
#     pickle.dump(myfield_review_step_eleven, f)

In [ ]:
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\rating_eleven_20012020.pkl'), 'wb') as f:
#     pickle.dump(ratings, f)

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[4,5,15,20,24,37,105,121,128,195,236,271,321,322,323,324,361,366,375,376,386,414,415,416,455,504,528,532,612,631,632,633,634,635,724,752,874,963,982,1031,1073,1119,1145,1164,1178,1226,1227,1228,1229,1230,1273,1282,1295,1296,1330,1348,1363,1464,1470,1473,1837,1838,1841,1928,2068,2073,2074,2076,2077,2091,2092,2094,2107,2123,2149,2180,2194,2243,2268,2277,2281,2283,2312,2315,2321,2327,2501,2534,2593,2636,2698,2738,2759,2761,2864,2875,3008,3021,3022,3027,3031,3034,3046,3047,3051,3056,3058,3061,3071,3072,3073,3074,3075,3076,3077,3080,3081,3084,3087,3089,3090,3091,3092,3093,3094,3095,3096,3119,3121,3134,3145,3190,3226,3227,3228,3229,3230,3231,3233,3285,3316,3332,3458,3519,3520,3521,3527,3586,3621,3666,3739,3759,3770,3787,3809,3832,3842,3938,3942,3960,4010,4039,4048,4081,4083,4120,4129,4190,4222,4223,4227,4284,4306,4317,4335,4429,4454,4464,4522,4531,4596,4629,4638,4642,4671,4697,4698,4737,4757,4788,4795,4920,4921,4926,4927,4928,4933,4963,4964,4976,4977,4995,874,1031,1079,1273,1932,2177,2269,2283,2336,2419,3455,3581,3740,4585,4834,612,874,1031,2281,2698,3008,14,18,22,44,66,84,146,149,166,171,207,244,312,322,323,324,353,412,433,455,456,537,586,592,593,624,635,705,735,747,843,932,937,993,1012,1031,1056,1072,1078,1144,1199,1229,1256,1331,1378,1484,1511,1512,1513,1514,1515,1516,1517,1518,1689,1739,1877,1912,1931,1936,1987,2039,2099,2104,2149,2154,2160,2172,2274,2285,2311,2335,2341,2345,2398,2404,2418,2446,2452,2544,2547,2599,2613,2615,2620,2622,2623,2631,2640,2650,2668,2670,2671,2672,2676,2680,2682,2718,2746,2774,2794,3121,3167,3174,3220,3221,3296,3320,3322,3345,3389,3425,3457,3470,3485,3486,3501,3518,3549,3556,3570,3580,3589,3592,3613,3659,3689,3730,3739,3758,3759,3772,3872,3874,3927,3947,3953,4008,4018,4036,4038,4120,4127,4148,4285,4297,4339,4404,4418,4419,4420,4447,4452,4472,4473,4476,4539,4549,4554,4646,4655,4715,4762,4763,4771,4786,4917,4926,4958,4963,4966,4988]
index_remove = list(dict.fromkeys(index_remove))
dataset = pd.read_pickle(os.path.join(os.getcwd(), "dataset_58,000_14012020_latest_version.pkl"))
dataset_eleven = dataset.iloc[50000:55000].reset_index(drop=True)
dataset_eleven = dataset_eleven[~dataset_eleven.index.isin(index_remove)]
dataset_eleven.shape

In [ ]:
"""
Loading the data extracted.
Those are the most important data source across all the notebook. 
The whole data extraction part was executed to end up with the following 6 files per mini-batch.

The examiner can directly import the files below to review the movie content extracted for the eleventh mini-batch.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\plot_eleven_20012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\rating_eleven_20012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\actors_eleven_20012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\director_eleven_20012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
        
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eleven\\reviews_eleven_24012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
assert len(plot) == len(rating) == len(actors) == len(director) == len(reviews)

In [ ]:
dataset_eleven['actors'] = actors
dataset_eleven['plot'] = plot
dataset_eleven['imdb_rating'] = rating
dataset_eleven['director'] = director
dataset_eleven['reviews'] = reviews
dataset_eleven = dataset_eleven.drop(["movieId", "imdbId", "synopsis_url"], axis=1)

In [ ]:
index_remove=[]
index_remove=[5,6,14,19,39,53,77,87,91,96,109,113,115,126,132,147,172,174,179,188,195,196,199,200,212,215,219,220,224,226,237,268,280,287,289,307,311,312,313,315,319,320,324,325,328,330,334,348,362,375,384,386,392,396,399,411,422,434,445,478,483,489,507,518,528,539,546,573,582,615,624,643,648,691,701,706,723,732,738,744,746,767,785,805,817,829,833,848,850,851,884,891,895,899,902,906,909,911,912,915,920,926,930,944,948,960,962,963,965,968,978,979,985,986,998,1000,1002,1009,1010,1032,1034,1037,1041,1055,1084,1092,1096,1097,1119,1145,1149,1152,1153,1157,1170,1171,1172,1173,1182,1195,1197,1199,1201,1211,1239,1252,1261,1264,1274,1282,1293,1299,1325,1330,1338,1339,1344,1358,1360,1367,1385,1387,1393,1395,1407,1411,1414,1418,1423,1432,1450,1455,1456,1461,1503,1507,1531,1536,1546,1548,1552,1601,1605,1606,1617,1630,1636,1637,1639,1651,1656,1667,1668,1686,1705,1731,1760,1761,1762,1763,1781,1795,1796,1797,1798,1799,1800,1801,1802,1803,1805,1806,1807,1814,1818,1819,1841,1856,1857,1859,1863,1877,1899,1902,1934,1951,1952,1956,1959,1960,1962,1963,1966,1969,1993,1994,2011,2014,2020,2026,2029,2030,2032,2036,2041,2045,2046,2047,2058,2059,2069,2077,2100,2101,2110,2113,2120,2130,2131,2132,2137,2159,2160,2169,2177,2201,2221,2254,2287,2324,2325,2330,2365,2369,2374,2382,2386,2388,2408,2439,2449,2451,2452,2458,2459,2460,2462,2463,2464,2467,2469,2470,2473,2477,2489,2490,2491,2492,2494,2498,2499,2500,2509,2519,2520,2526,2528,2535,2537,2542,2544,2546,2547,2551,2558,2576,2577,2582,2583,2584,2589,2592,2593,2602,2606,2610,2611,2615,2618,2620,2629,2765,2894,2897,2900,2902,2904,2910,2920,2929,2933,2937,2942,2943,2944,2946,2955,2956,2959,2960,2964,2977,2987,2990,3000,3069,3075,3082,3084,3095,3097,3098,3099,3101,3103,3106,3121,3122,3138,3140,3163,3166,3173,3175,3179,3183,3187,3194,3197,3211,3212,3229,3230,3244,3262,3266,3267,3271,3286,3293,3294,3302,3305,3308,3318,3319,3320,3322,3328,3335,3340,3342,3367,3386,3389,3391,3395,3404,3405,3409,3414,3416,3421,3424,3427,3437,3441,3449,3451,3454,3472,3479,3487,3496,3502,3532,3546,3585,3586,3587,3590,3594,3597,3606,3608,3609,3613,3621,3633,3644,3647,3655,3659,3660,3671,3678,3688,3718,3725,3726,3728,3738,3751,3774,3781,3792,3816,3820,3833,3836,3860,3870,3892,3897,3904,3921,3939,3944,3945,3963,3972,3976,3977,4003,4013,4023,4024,4040,4042,4047,4079,4080,4085,4093,4095,4096,4098,4099,4100,4104,4105,4106,4109,4114,4116,4119,4122,4123,4128,4142,4144,4145,4146,4147,4149,4172,4181,4199,4203,4206,4209,4212,4213,4215,4217,4237,4238,4249,4251,4252,4258,4275,4279,4283,4286,4287,4296,4302,4305,4306,4310,4314,4324,4326,4332,4333,4339,4347,4349,4383,4385,4386,4390,4403,4413,4425,4429,4430,4431,4461,4470,4487,4491,4508,4516,4529,4545,4558,4560,4561,4563,4565,4571,4578,4588,4596,4597,4618,4620,4621,4625]
dataset_eleven = dataset_eleven[~dataset_eleven.index.isin(index_remove)]
dataset_eleven = dataset_eleven[dataset_eleven.astype(str)['reviews'] != '[]']
dataset_eleven.shape

In [ ]:
"""
This is the table to use in Part 1, and the table that will be concatenated with the rest of the mini-batch tables.
"""
# dataset_eleven.to_pickle(os.path.join(os.getcwd(), "58,000 movies\\movie_eleven\\dataset_eleven_final_25012020.pkl"))